# ACS Data Cleaning and Exploration

### Importing Required Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
from varname import nameof
%load_ext google.cloud.bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '../zori-data-extr-be793d5c3325.json'

# Set your default project here
pandas_gbq.context.project = 'bigquery-public-data'
pandas_gbq.context.dialect = 'standard'

## Using Google BigQuery to Download ACS Data

In [79]:
%%bigquery --use_rest_api ACS_2018
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2018_5yr`

Downloading: 100%|██████████| 33120/33120 [00:04<00:00, 6910.54rows/s] 


In [71]:
%%bigquery --use_rest_api ACS_2017
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2017_5yr`

Downloading: 100%|██████████| 33120/33120 [00:05<00:00, 6020.01rows/s] 


In [17]:
%%bigquery --use_rest_api ACS_2016
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2016_5yr`

Downloading: 100%|██████████| 33120/33120 [00:05<00:00, 6011.01rows/s] 


In [18]:
%%bigquery --use_rest_api ACS_2015
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2015_5yr`

Downloading: 100%|██████████| 33120/33120 [00:08<00:00, 3826.22rows/s]


In [19]:
%%bigquery --use_rest_api ACS_2014
SELECT *
FROM `bigquery-public-data.census_bureau_acs.zip_codes_2014_5yr`

Downloading: 100%|██████████| 33120/33120 [00:05<00:00, 6009.81rows/s] 


## ACS Data Cleaning

In [62]:
pd.set_option('display.max_columns', None)
#Looking at the shape of each dataframe
print(ACS_2018.shape)
print(ACS_2017.shape)
print(ACS_2016.shape)
print(ACS_2015.shape)
print(ACS_2014.shape, '\n')

(33120, 240)
(33120, 252)
(33120, 252)
(33120, 247)
(33120, 252) 



Based on the data shown above, a number of columns are missing from the 2018 data set. Since the 2018 data set is the latest, all additional columns seen in prior year surveys will be dropped for consistency.

In [72]:
def clean_columns(df_year1, df_year2):
    """
    This function cleans the second dataframe to only include columns of the first dataframe.
    
    Arguments:
    
    df_year1 : (Pandas dataframe) dataframe containing the columns of interest
    df_year2 : (Pandas dataframe) dataframe containing the columns of interest + additional
    
    Returns
    df_year2 : (Pandas dataframe) modified version of the df_year2 dataframe from inputs
    
    """
    diff = np.setdiff1d(df_year2.columns, df_year1.columns)
    print(f'Columns from {nameof(df_year2)} non included in {nameof(df_year1)}:\n\nTotal of {len(diff)}\n\n{diff}\n\n')
    print('Removing columns...')
    
    #Dropping the columns from df_year2 that are not present in the df_year1
    df_year2 = df_year2.drop(columns = diff, inplace = True)
    
    print('Process complete\n\n\n')# Results:\n\nTotal number of columns in {nameof(df_year1)}: {df_year1.shape[1]}\n')
#     print(f'Total number of columns in {nameof(df_year2)}: {df_year2.shape[1]}')
    
    return df_year2

In [73]:
#Using the above defined function to clean columns
clean_columns(ACS_2018, ACS_2017)
clean_columns(ACS_2018, ACS_2016)
clean_columns(ACS_2018, ACS_2015)
clean_columns(ACS_2018, ACS_2014)

Columns from df_year2 non included in df_year1:

Total of 12

['amerindian_including_hispanic' 'asian_including_hispanic'
 'black_including_hispanic' 'commute_35_39_mins' 'commute_40_44_mins'
 'commute_5_9_mins' 'commute_60_89_mins' 'commute_90_more_mins'
 'households_retirement_income' 'male_60_61' 'male_62_64'
 'white_including_hispanic']


Removing columns...
Process complete
Columns from df_year2 non included in df_year1:

Total of 12

['amerindian_including_hispanic' 'asian_including_hispanic'
 'black_including_hispanic' 'commute_35_39_mins' 'commute_40_44_mins'
 'commute_5_9_mins' 'commute_60_89_mins' 'commute_90_more_mins'
 'households_retirement_income' 'male_60_61' 'male_62_64'
 'white_including_hispanic']


Removing columns...
Process complete
Columns from df_year2 non included in df_year1:

Total of 13

['amerindian_including_hispanic' 'asian_including_hispanic'
 'black_including_hispanic' 'commute_35_39_mins' 'commute_40_44_mins'
 'commute_5_9_mins' 'commute_60_89_mins' 'co

In [74]:
#Looking at the shape of each dataframe to confirm column cleaning has been done appropriately
print(ACS_2018.shape)
print(ACS_2017.shape)
print(ACS_2016.shape)
print(ACS_2015.shape)
print(ACS_2014.shape, '\n')

(33120, 240)
(33120, 240)
(33120, 240)
(33120, 234)
(33120, 240) 



In [94]:
np.setdiff1d(ACS_2018.columns, ACS_2015.columns)

array(['pop_15_and_over', 'pop_divorced', 'pop_never_married',
       'pop_now_married', 'pop_separated', 'pop_widowed'], dtype=object)

As can be seen above for year 2015 there are less columns than in any other year.

In [83]:
#Adding a year column to each survey dataframe
ACS_2018['year'] = 2018
ACS_2017['year'] = 2017
ACS_2016['year'] = 2016
ACS_2015['year'] = 2015
ACS_2014['year'] = 2014

#Combining all ACS dataframes to one global ACS dataframe
ACS = pd.concat([ACS_2018, ACS_2017, ACS_2016, ACS_2015, ACS_2014], ignore_index = True)

### Dropping Income Columns

Partly Parrots will be looking at IRS data for everything income related, as such all income related columns will be dropped.

In [97]:
ACS = ACS[ACS.columns.drop(list(ACS.filter(regex = 'income')))]

### Looking at Missingness

In [122]:
def missingness(df):
    """
    This function looks at the number of missing values in a dataframe
    
    Arguments:
    
    df          : (Pandas dataframe) dataframe of interest
    
    Returns
    
    missing_col : (dictionary) column name and number of missing values 
    
    """
    all_cols_miss = df.isna().mean().sort_values(ascending = False)

    missing_col_list = []
    missing_col = {}

    for i in range(all_cols_miss[all_cols_miss > 0].shape[0]):
        missing_col_list.append([all_cols_miss[all_cols_miss > 0].index[i], \
                                round(all_cols_miss[all_cols_miss > 0][i], 4)])

    missing_col.update(missing_col_list)
    return missing_col

In [111]:
#Looking at missing values in data

{'pop_divorced': 132480,
 'pop_widowed': 132480,
 'pop_separated': 132480,
 'pop_now_married': 132480,
 'pop_never_married': 132480,
 'pop_15_and_over': 132480,
 'pop_5_years_over': 99360,
 'speak_spanish_at_home': 99360,
 'speak_only_english_at_home': 99360,
 'speak_spanish_at_home_low_english': 99360,
 'aggregate_travel_time_to_work': 64540,
 'median_rent': 29439,
 'renter_occupied_housing_units_paying_cash_median_gross_rent': 28686,
 'owner_occupied_housing_units_lower_value_quartile': 13304,
 'owner_occupied_housing_units_upper_value_quartile': 11229,
 'owner_occupied_housing_units_median_value': 11200,
 'median_income': 9566,
 'median_year_structure_built': 4577,
 'gini_index': 4431,
 'median_age': 2667,
 'different_house_year_ago_different_city': 690,
 'bachelors_degree_2': 690,
 'high_school_including_ged': 690,
 'less_than_high_school_graduate': 690,
 'different_house_year_ago_same_city': 690,
 'graduate_professional_degree': 690,
 'some_college_and_associates_degree': 690,
 'p

##### Taking a closer look into the high missingness columns:

In [118]:
ACS = ACS[ACS['worked_at_home'].notna()]

In [123]:
ACS.shape

(165565, 223)

In [124]:
missingness(ACS)

{'pop_15_and_over': 0.8,
 'pop_never_married': 0.8,
 'pop_divorced': 0.8,
 'pop_separated': 0.8,
 'pop_now_married': 0.8,
 'pop_widowed': 0.8,
 'pop_5_years_over': 0.5999,
 'speak_spanish_at_home': 0.5999,
 'speak_spanish_at_home_low_english': 0.5999,
 'speak_only_english_at_home': 0.5999,
 'aggregate_travel_time_to_work': 0.3896,
 'median_rent': 0.1776,
 'renter_occupied_housing_units_paying_cash_median_gross_rent': 0.1731,
 'owner_occupied_housing_units_lower_value_quartile': 0.0803,
 'owner_occupied_housing_units_upper_value_quartile': 0.0678,
 'owner_occupied_housing_units_median_value': 0.0676,
 'median_income': 0.0576,
 'median_year_structure_built': 0.0276,
 'gini_index': 0.0266,
 'median_age': 0.0161,
 'bachelors_degree_2': 0.004,
 'not_us_citizen_pop': 0.004,
 'graduate_professional_degree': 0.004,
 'population_1_year_and_over': 0.004,
 'less_than_high_school_graduate': 0.004,
 'high_school_including_ged': 0.004,
 'different_house_year_ago_different_city': 0.004,
 'different_h

In [125]:
ACS.head()

,geo_id,do_date,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,male_15_to_17,male_18_to_19,male_20,male_21,male_22_to_24,male_25_to_29,male_30_to_34,male_35_to_39,male_40_to_44,male_45_to_49,male_50_to_54,male_55_to_59,male_65_to_66,male_67_to_69,male_70_to_74,male_75_to_79,male_80_to_84,male_85_and_over,female_under_5,female_5_to_9,female_10_to_14,female_15_to_17,female_18_to_19,female_20,female_21,female_22_to_24,female_25_to_29,female_30_to_34,female_35_to_39,female_40_to_44,female_45_to_49,female_50_to_54,female_55_to_59,female_60_to_61,female_62_to_64,female_65_to_66,female_67_to_69,female_70_to_74,female_75_to_79,female_80_to_84,female_85_and_over,white_pop,population_1_year_and_over,population_3_years_over,pop_5_years_over,pop_15_and_over,pop_16_over,pop_25_years_over,pop_25_64,pop_never_married,pop_now_married,pop_separated,pop_widowed,pop_divorced,not_us_citizen_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,hispanic_any_race,not_hispanic_pop,asian_male_45_54,asian_male_55_64,black_male_45_54,black_male_55_64,hispanic_male_45_54,hispanic_male_55_64,white_male_45_54,white_male_55_64,median_income,pop_determined_poverty_status,poverty,gini_index,housing_units,renter_occupied_housing_units_paying_cash_median_gross_rent,owner_occupied_housing_units_lower_value_quartile,owner_occupied_housing_units_median_value,owner_occupied_housing_units_upper_value_quartile,occupied_housing_units,housing_units_renter_occupied,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,dwellings_1_units_detached,dwellings_1_units_attached,dwellings_2_units,dwellings_3_to_4_units,dwellings_5_to_9_units,dwellings_10_to_19_units,dwellings_20_to_49_units,dwellings_50_or_more_units,mobile_homes,housing_built_2005_or_later,housing_built_2000_to_2004,housing_built_1939_or_earlier,median_year_structure_built,married_households,nonfamily_households,family_households,households_public_asst_or_food_stamps,male_male_households,female_female_households,children,children_in_single_female_hh,median_rent,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,different_house_year_ago_different_city,different_house_year_ago_same_city,families_with_young_children,two_parent_families_with_young_children,two_parents_in_labor_force_families_with_young_children,two_parents_father_in_labor_force_families_with_young_children,two_parents_mother_in_labor_force_families_with_young_children,two_parents_not_in_labor_force_families_with_young_children,one_parent_families_with_young_children,father_one_parent_families_with_young_children,father_in_labor_force_one_parent_families_with_young_children,commute_less_10_mins,commute_10_14_mins,commute_15_19_mins,commute_20_24_mins,commute_25_29_mins,commute_30_34_mins,commute_35_44_mins,commute_60_more_mins,commute_45_59_mins,commuters_16_over,walked_to_work,worked_at_home,no_car,no_cars,one_car,two_cars,three_cars,four_more_cars,aggregate_travel_time_to_work,commuters_by_public_transportation,commuters_by_bus,commuters_by_car_truck_van,commuters_by_carpool,commuters_by_subway_or_elevated,commuters_drove_alone,group_quarters,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,less_than_high_school_graduate,high_school_including_ged,bachelors_degree_2,bachelors_degree_or_higher_25_64,graduate_professional_degree,some_college_and_associates_degree,male_45_64_associates_degree,male_45_64_bachelors_degree,male_45_64_graduate_degree,male_45_64_less_than_9_grade,male_45_64_grade_9_12,male_45_64_high_school,male_45_64_some_college,male_45_to_64,employed_pop,unemployed_pop,pop_in_labor_force,not_in_labor_force,workers_16_a

In [ ]:
zri = pd.read_csv('../Da')